In [1]:
import h5py
import pyccl
import scipy
import numpy
import json

In [2]:
# Load in Data

PATH = '/pscratch/sd/y/yhzhang/TensorCloud/'
DATA_PATH = PATH + 'DATA/'
CODE_PATH = PATH + 'CODE/'
PLOT_PATH = PATH + 'PLOT/'

BIN_SIZE = 4
GRID_SIZE = 72
LATENT_SIZE = 32
DATA_SIZE = 100000
COLOR = ['blue', 'green', 'orange', 'red']

with h5py.File(DATA_PATH + 'DATA.hdf5','r') as FILE:
    
    Z_GRID = numpy.array(FILE['realizations/z'][:GRID_SIZE + 1], dtype = 'double')
    GRID = numpy.array(FILE['realizations/pdfs'][:, :BIN_SIZE, :GRID_SIZE + 1], dtype = 'double')

In [3]:
# COSMOLOGY

with open(DATA_PATH + 'COSMO.json', 'r') as FILE:
    COSMO = json.load(FILE)
    
COSMO_CCL = pyccl.cosmology.Cosmology(
    h=COSMO['H'],
    w0=COSMO['W0'],
    wa=COSMO['WA'],
    n_s=COSMO['NS'],
    m_nu=COSMO['MNU'],
    Neff=COSMO['NEFF'],
    T_CMB=COSMO['TCMB'],
    mass_split='single',
    sigma8=COSMO['SIGMA8'],
    Omega_k=COSMO['OMEGAK'],
    Omega_c=COSMO['OMEGAC'],
    Omega_b=COSMO['OMEGAB'],
    mg_parametrization=None,
    matter_power_spectrum='halofit',
    transfer_function='boltzmann_camb',
    extra_parameters={
        'camb': {'kmax': 10000, 'lmax': 10000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

In [4]:
# PMM, matter-matter-power-spectra

PSI_GRID = numpy.mean(GRID, axis = 0)
SIGMA_GRID = numpy.std(GRID, axis = 0)
A_GRID = numpy.array(1 / (1 + Z_GRID))
CHI_GRID = COSMO_CCL.comoving_radial_distance(a = A_GRID)
PHI_GRID = PSI_GRID * COSMO_CCL.h_over_h0(a = 1 / (1 + Z_GRID)) * COSMO['H'] * 100000 / scipy.constants.c

ELL_SIZE = 100
ELL_DATA = numpy.logspace(1, 4, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_DATA)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where = CHI_MESH > 0))

PMM_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):

    PMM_GRID[:,GRID_INDEX] = COSMO_CCL.linear_matter_power(k = SCALE_GRID[:,GRID_INDEX], a = A_GRID[GRID_INDEX])

In [5]:
#numerical integral

def integralI2(chi1, chi2, chi3, power1, power2, redshift1, redshift2):

    def formulan(chi):
        
        result = (chi2**2 - chi**2) / 2 / (chi2 - chi1) - chi * chi2 * numpy.log(chi2 / chi) / (chi2 - chi1)

        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))

        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)**2

        return result

    def formula0(chi):
        
        result = (chi2**2 - chi**2) / 2 / (chi2 - chi1) - chi * chi2 * numpy.log(chi2 / chi) / (chi2 - chi1)

        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2**2 - 2 * chi1 * chi2 + chi**2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))

        result = result * (((chi/chi2)**3)*power2)
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2)**2

        return result  
    
    if chi1 == 0:
        
        integral, error = scipy.integrate.quad_vec(f = formula0, a = chi1, b = chi2)
        
    else:
        
        integral, error = scipy.integrate.quad_vec(f = formulan, a = chi1, b = chi2)

    return integral

In [6]:
#coefficient equation

def coefficientJ2(chi1, chi2, chi3, power1, power2, redshift1, redshift2):

    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    y = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)

    if a == 1:

        formula = (b * (6642 + z * ( - 6669 + 1867 * z) + 18 * b * (294 + z * ( - 308 + 89 * z))) - 45 * (1 + b) * (112 + z * ( - 104 + 27 * z)) * numpy.log(1 + b)) / (6350400 * b)

    else:

        formula = - ((1 / (4 * a**5 * b)) * ((1 / 15) * ( - 1 + a)**4 * b * (5 * a**2 * (y + a * ( - 4 + 3 * y)) - 2 * a * (2 * y + a * ( - 5 + 6 * y + 3 * a * ( - 5 + 4 * y))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a**2) + y + a * (3 + 2 * a * (3 + 5 * a)) * y) * z**2) * numpy.log(1 - a)**2 + 
(1 / 3150) * (( - 1 + a)**2 * numpy.log(1 - a) * (b * (35 * a**2 * ( - 5 * a * (2 + a * (23 + a * ( - 52 + 9 * a) - 18 * b)) + (7 + a * (28 + a * (19 + 36 * ( - 5 + a) * a - 60 * b) - 30 * b)) * y) + 14 * a * (5 * a * (7 + a * (28 + a * (19 + 36 * ( - 5 + a) * a - 60 * b) - 30 * b)) - 26 * y + a * ( - 56 + 2 * a * ( - 13 + 3 * a * (4 + (114 - 25 * a) * a)) + 
75 * (1 + a * (2 + 3 * a)) * b) * y) * z + (7 * a * ( - 26 - 2 * a * (28 + a * (13 + 3 * a * ( - 4 + a * ( - 114 + 25 * a)))) + 75 * a * (1 + a * (2 + 3 * a)) * b) + 
(141 + a * (201 + 51 * a - 169 * a**2 - 424 * a**3 - 3850 * a**4 + 900 * a**5 - 315 * (1 + a * (2 + a * (3 + 4 * a))) * b)) * y) * z**2) - 
210 * ( - 1 + a) * a * (1 + b) * (5 * a**2 * (y + a * ( - 4 + 3 * y)) - 2 * a * (2 * y + a * ( - 5 + 6 * y + 3 * a * ( - 5 + 4 * y))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a**2) + y + a * (3 + 2 * a * (3 + 5 * a)) * y) * z**2) * numpy.log(1 + b))) - 
(1 / 1323000) * (a * (b * (245 * a**2 * ( - 420 * y + 20 * a**2 * ( - 60 + 45 * b * ( - 6 + y) + 62 * y) + 9 * a**5 * ( - 195 + 148 * y) + 5 * a**3 * ( - 830 + 60 * b * (27 - 16 * y) + 193 * y) + 
150 * a * (4 + (5 + 12 * b) * y) + 2 * a**4 * (3725 - 2322 * y + 225 * b * ( - 4 + 3 * y))) - 
14 * a * (35 * a * (420 + a * ( - 150 * (5 + 12 * b) + a * ( - 20 * (62 + 45 * b) + a * ( - 965 + 4800 * b - 18 * a * ( - 258 + 74 * a + 75 * b))))) + 
( - 10920 + 2 * a * (9030 + a * (10360 + a * (7805 + 6 * a * (1008 + a * ( - 9688 + 3125 * a))))) + 525 * a * (60 + a * (30 + a * (20 + 9 * a * ( - 25 + 8 * a)))) * b) * y) * z + (7 * a * (10920 - 2 * a * (9030 + a * (10360 + a * (7805 + 6 * a * (1008 + a * ( - 9688 + 3125 * a))))) - 
525 * a * (60 + a * (30 + a * (20 + 9 * a * ( - 25 + 8 * a)))) * b) + ( - 59220 + a * (92610 + 85470 * a + 62685 * a**2 + 48111 * a**3 + 38584 * a**4 - 629500 * a**5 + 219375 * a**6 + 2205 * (60 + a * (30 + a * (20 + a * (15 + 4 * a * ( - 72 + 25 * a))))) * b)) * y) * z**2) + 
210 * a * (1 + b) * ( - 210 * a * (y * ( - 8 + z) - 4 * z) * z - 420 * y * z**2 - 35 * a**3 * (30 * ( - 8 + y) + 4 * (15 - 4 * y) * z + ( - 8 + 3 * y) * z**2) + 
14 * a**4 * ( - 1500 + 850 * y + 5 * (340 - 117 * z) * z + 6 * y * z * ( - 195 + 74 * z)) - 140 * a**2 * ( - 3 * ( - 10 + z) * z + y * (15 + ( - 6 + z) * z)) + 
30 * a**6 * (4 * y * (21 + 5 * z * ( - 7 + 3 * z)) - 7 * (15 + 2 * z * ( - 12 + 5 * z))) - 7 * a**5 * ( - 2200 + 18 * (175 - 68 * z) * z + y * (1575 + 8 * z * ( - 306 + 125 * z)))) * numpy.log(1 + b)))))

    coefficient = chi2**3 * power2 * (1 + redshift2)**2 * formula
    
    return coefficient

In [7]:
#Case1 test: For chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

1461.30684999374 1497.8183354014277 1534.1133113296617 49919.45246058225 48958.62248873209 0.36 0.37


In [8]:
#Case1 test: For chi1 > 0

INTEGRAL = integralI2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(89475.08010474301, 89475.13185852158, 5.784155598931306e-07)

In [9]:
#Case2 test: For chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = PMM_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = PMM_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.42674493673601 88.64121385461215 0.0 2114.2567965927065 0.0 0.01


In [10]:
#Case2 test: For chi1 = 0

INTEGRAL = integralI2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficientJ2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1

(145116.8295688111, 145116.82956883652, 1.751931932858497e-13)